# Floating Cable Trap Calculator

Purdue University

**Prerequisites (pip install or conda install)**
* For Computation
    * numpy
    * matplotlib
    * ipywidgets
    * mplcursors
* For Rendering
    * ViewSCAD
    * SolidPython
    * OpenSCAD (Download executable from [https://openscad.org/](https://openscad.org/))

Users of **JupyterLab** should follow the instructions of [https://github.com/nickc92/ViewSCAD](ViewSCAD) to successfully render the 3D files. **Jupyter Notebook** users should be okay.

Set the value of `plot_size` below, and then run all cells (`Run > Run All`).
Adjust the values in the widgets, and the plot automatically updates.

In [40]:
plot_size = (25, 15) # (width, height) in cm

**************************************************
There is no need to edit any codes below.

In [41]:
# Imports packages
import math
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker
import mplcursors # For interactive data points

# For 3D generation
import viewscad
from solid import *
from solid.utils import *  # Not required, but the utils module is useful

In [42]:
# Initialize Program, Run Once
w = dict()

# Universal constants
epsilon_0 = 8.85e-12
mu_0 = 4 * math.pi * 1e-7
c_0 = 299792458

# Initialize variables
k=0
mu_r=0
a=0
ct=0
l=0
f_target = 0
epsilon = epsilon_0
mu = mu_0
a = 0
C_T_limit = []
C_T = []
fig = []
ax = []
lengths=[]
b_arr=[]

# Define certain widgets
out = widgets.Output(layout={'border': '1px solid black'})
out.layout.height = '100%'

file_name = widgets.Text( # Plot file name
    value='file_name.png',
    placeholder='Type something',
    description='File Name:',
    disabled=False   
)
file_name_csv = widgets.Text( # Plot file name
    value='file_name.csv',
    placeholder='Type something',
    description='File Name:',
    disabled=False   
)

# Define helper functions
def signif(x, p):
    x = np.asarray(x)
    x_positive = np.where(np.isfinite(x) & (x != 0), np.abs(x), 10**(p-1))
    mags = 10 ** (p - 1 - np.floor(np.log10(x_positive)))
    return np.round(x * mags) / mags

# Save values
def save_constants():
    global f_target, epsilon, mu, a
    f_target = w['f'].value * 1000000
    epsilon =  w['k'].value * epsilon_0
    mu = w['mur'].value * mu_0
    a = w['a'].value / 100

def save_constraints_CT():
    global length, C_T_limit, C_T
    length = np.array(w['l'].value) / 100
    C_T_limit = np.array(w['ct'].value) * 1E-12
    C_T = np.linspace(C_T_limit[0], C_T_limit[1], 100)

def update_values():
    global out, fig, ax, file_name, lengths, b_arr, file_name_csv
    save_constants()
    save_constraints_CT()
    with out: 
        out.clear_output()
        print(f"f={w['f'].value} MHz, K={w['k'].value}, mu_r={w['mur'].value}, a={w['a'].value} cm")
        print(f"Plotting values of C_T between {w['ct'].value[0]}-{w['ct'].value[1]} pF")
        print(f"and lengths between {w['l'].value[0]}-{w['l'].value[1]} cm") 
        print("Optimal design is at maximum b")
        fig, ax, lengths, _, b_arr = plot_b_CT()
        print("Legend - Trap Lengths (cm):")
        print(np.linspace(length[0], length[1],w['n(l)'].value)*100)
    file_name.value = f"a={w['a'].value}_ct={w['ct'].value[0]}-{w['ct'].value[1]}_len={w['l'].value[0]}-{w['l'].value[1]}_K={w['k'].value}.png"
    file_name_csv.value = file_name.value[:-3] + "csv"
# Plot
def plot_b_CT():
    # b vs CT for multiple lengths
    global f_target, C_T, mu, epsilon, length
    b_arr = []
    lengths = np.linspace(length[0], length[1],w['n(l)'].value)
    fig, ax = plt.subplots(1, 1, figsize=(plot_size[0]/2.54, plot_size[1]/2.54))
    for l in lengths:
        b_arr.append( a * np.exp((1/(2*math.pi*f_target**2*l*C_T*mu)) - ((2*math.pi*l*epsilon)/(C_T))))
        ax.plot(C_T*1E12, b_arr[-1]*100, label=str(signif(l*100, 2))+" cm")
    
    plt.grid(True, which="both", ls="--")
    plt.xlabel('$C_T$ Tuning Capacitance (pF)')
    plt.ylabel('$b$ Outer Radius (cm)')
    plt.legend(bbox_to_anchor=(1, 0), loc="lower right",
                bbox_transform=fig.transFigure, ncol=1)
    plt.legend(bbox_to_anchor=(1.03, 0.5), loc="center left", borderaxespad=0, title = "Trap length")
    plt.tight_layout(rect=[0, 0, 0.95, 1])
    mplcursors.cursor(hover=True)
    plt.show()
    return fig, ax, lengths, C_T, b_arr
def save_plot():
    plt.savefig('filename.ext', bbox_inches='tight')

In [43]:
# Run this block once, and adjust parameters in the output. 
# Values update automatically, No need to run again. 
%matplotlib widget

# Design Constants
w['f'] = widgets.BoundedFloatText(
    value=128,
    min=50,
    max=500,
    step=1,
    description='f in MHz',
    disabled=False
)
w['k'] = widgets.BoundedFloatText(
    value=2.0,
    min=0,
    max=10.0,
    step=0.01,
    description='K',
    disabled=False
)
w['mur'] = widgets.BoundedFloatText(
    value=1.0,
    min=0,
    max=5.0,
    step=0.001,
    description='mu_r',
    disabled=False
)
w['a'] = widgets.BoundedFloatText(
    value=0.95,
    min=0,
    max=50,
    step=0.01,
    description='a in cm',
    disabled=False
)
print("Plot updates automatically")
print("\u001b[31mConstants\u001b[30m")
print('Input: Larmor Frequency in MHz')
display(w['f'])
print('Input: Dielectric Constant of Core Material/Substrate')
display(w['k'])
print('Input: Relative Permeability of Core Material/Substrate')
print("\t(Hint: It is usually very close to = 1 for non-ferromagnetic materials)")
display(w['mur'])
print('Input: Inner Diameter (cm)\n\t(Hint: Inner diameter should closely-fit the cables)')
display(w['a'])

# Design Constraints
print("\u001b[31mConstraints\u001b[30m")
print("Design Constraints\n\tYou can type the number or drag the slider bar\n","-"*60)

w['ct'] = widgets.FloatRangeSlider(
    value=[50, 300],
    min=0,
    max=1000.0,
    step=0.001,
    description='C_T in pF',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

w['l'] = widgets.FloatRangeSlider(
    value=[6, 15],
    min=0,
    max=30.0,
    step=0.001,
    description='L in cm',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

w['n(l)'] = widgets.IntSlider(
    value=10,
    min=0,
    max=50,
    step=1,
    description='num(length)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

print("Input: Range of Tuning Capacitances that your lab has (in pF picoFarads)")
print("\t(Hint: You can tune using capacitors in series. Input the total acceptable capacitance)")
display(w['ct'])

print("\nInput: Range of acceptable cable trap length (in cm)")
display(w['l'])

print("\nInput: Number of lengths to plot")
display(w['n(l)'])

# Reset plot
print("Update graph (if not auto updated)")
force_update = widgets.Button(
    description='Update Graph',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    icon='warning' # (FontAwesome names without the `fa-` prefix)
)
display(force_update)
force_update.on_click(lambda b : update_values())

# Detects changes in parameters and saves
for i in w:
    w[i].observe(lambda b: update_values(), 'value')
display(out)
with out: update_values() # Initializes variables



Plot updates automatically
Constants
Input: Larmor Frequency in MHz


BoundedFloatText(value=128.0, description='f in MHz', max=500.0, min=50.0, step=1.0)

Input: Dielectric Constant of Core Material/Substrate


BoundedFloatText(value=2.0, description='K', max=10.0, step=0.01)

Input: Relative Permeability of Core Material/Substrate
	(Hint: It is usually very close to = 1 for non-ferromagnetic materials)


...


Input: Number of lengths to plot


IntSlider(value=10, continuous_update=False, description='num(length)', max=50)

Update graph (if not auto updated)


Button(description='Update Graph', icon='warning', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [44]:
save_button = widgets.Button(
    description='Save plot',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='save' # (FontAwesome names without the `fa-` prefix)
)

print("Save plot: Supports images (jpg/png/tif), pdf, svg")
display(file_name)
display(save_button)
save_button.on_click(lambda b : fig.savefig(file_name.value))



Save plot: Supports images (jpg/png/tif), pdf, svg


Text(value='a=0.95_ct=50.0-300.0_len=6.0-15.0_K=2.0.png', description='File Name:', placeholder='Type somethin…

Button(button_style='success', description='Save plot', icon='save', style=ButtonStyle())

In [45]:
def save_csv():
    global C_T, b_arr, lengths, w, file_name_csv
    raw_data = np.transpose(np.vstack((C_T, np.array(b_arr))))
    # Includes parameters in csv
    comments = ""
    for i in w:
        comments = comments + str(i) + "=" + str(w[i].value) + ", "
    comments = comments + " -- Columns=(C_T, b, b, b, ...), b for each length --"
    # Header
    headers = "CT"
    for l in lengths:
        headers = headers + "," + str(l)
    np.savetxt(file_name_csv.value, raw_data, delimiter= ',', header = headers, footer = comments)

print("Save raw data as csv:")
save_csv_button = widgets.Button(
    description='Save csv',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='save' # (FontAwesome names without the `fa-` prefix)
)
save_csv_button.on_click(lambda b : save_csv())
display(file_name_csv)
display(save_csv_button)

Save raw data as csv:


Text(value='a=0.95_ct=50.0-300.0_len=6.0-15.0_K=2.0.csv', description='File Name:', placeholder='Type somethin…

Button(button_style='success', description='Save csv', icon='save', style=ButtonStyle())

In [46]:
outer_radius = 20
length = 100
inner_radius = 9.5
hole_rad = 2.5
head_height = 2.75*1.1
head_diam = 9.5
num_hole = 2

d = difference() (
  cylinder(r=outer_radius, h=length),
  
  translate([(outer_radius+length)*2,0,0])(
      cube((outer_radius+length)*4, center=True)
  ),
      translate([0,0,-1])(
      cylinder(r=inner_radius, h=length*2)
      )
)



for i in range(num_hole):
    x = i*length/num_hole + length/(2*num_hole)
    screw1 = translate([outer_radius-head_height-math.sqrt(3)*outer_radius,0,x])(rotate([0,90,0])(translate([0,(outer_radius + inner_radius) / 2,0])(translate([0,0,head_height])(cylinder(r=hole_rad, h=outer_radius)) + translate([0,0,-head_height])(cylinder(d=head_diam, h=head_height*2)))))
    screw2 = translate([outer_radius-head_height-math.sqrt(3)*outer_radius,0,x])(rotate([0,90,0])(translate([0,-(outer_radius + inner_radius) / 2,0])(translate([0,0,head_height])(cylinder(r=hole_rad, h=outer_radius)) + translate([0,0,-head_height])(cylinder(d=head_diam, h=head_height*2)))))
    d = d - (screw1 + screw2)

r = viewscad.Renderer()
r.render(d)


print("Use your mouse to rotate and zoom. Drag rotates the view; Control + drag translates the view.")
print("Units: mm")

# If this line fails to run, it means Python could not find your openSCAD executable.
# Edit this line to include your openscad executable. Check viewSCAD documentation.
#     e.g.  renderer = viewscad.Renderer(openscad_exec='path_of_my_openscad')
# stl_file_name = file_name.value[:3]+"stl"
# r = viewscad.Renderer(outfile=stl_file_name)
# r.render(d)

Geometries in cache: 9
Geometry cache size in bytes: 22680
CGAL Polyhedrons in cache: 12
CGAL cache size in bytes: 1822624
Total rendering time: 0:00:00.531
Top level object is a 3D object:
   Simple:        yes
   Vertices:      308
   Halfedges:     924
   Edges:         462
   Halffacets:    304
   Facets:        152
   Volumes:         2


Use your mouse to rotate and zoom. Drag rotates the view; Control + drag translates the view.
Units: mm
